# Rapport TP1 Métaheuristiques d'optimization 

### Introduction:

Ce TP consiste à comparer la performance et l'efficacité de trois métaheuristiques du type Monte-Carlo. 
J'ai essayé de regrouper tout le code dans un seul notebook même si le développement de chaque algorithme est fait dans un fichier séparé accompagnant ce rapport. 
Ce notebook représente le rapport du travail et comporte tout le code source et le résultat de l'exécution avec des commentaires et des explications.
Pour les deux algorithmes du hill climbing et du recuit simulé, il se peut que l'algorithme rentre dans une boucle de très longue durée lors de la génération du nouveau candidat à cause des contraintes, il serait utile donc d'interrompre le Kernel et réexécuter l'appel de l'algorithme.

Deux bibliothèques seulement sont utilisées pour ce travail :

In [1046]:
import numpy as np
import pandas as pd
#from tqdm import tqdm 

La fonction 'bornes' permet de retourner les bornes des variables du problème et parce que les bornes des variables x3 et x4 dépendent des valeurs de z1 et z2, on doit calculer les bornes à chaque fois en fonction du nouveau point qu'on a.

Ces bornes n'étaient pas explicites, il fallait donc simplifier les différentes inégalités qu'on a dans le problème. Dans ce qui suit la démonstration :

•	x1 <= 99 : c'est une donnée, mais j'ai choisi comme borne inferieure 1 après plusieurs exécutions ou l'algorithme tournait dans une boucle très longue pour trouver un candidat acceptable.

•	x2 >= 1 : donnée, et j'ai choisi comme borne supérieure 100 pour la même raison

•	0.00954*x3 <= z2 & 0.0193*x3 <= z1 & x3 <= 200 ce qui implique que x3 <= min(0.625*x[1]/0.00954,0.625*x[0]/0.0193, 200), et j'ai choisi comme borne inferieure 1

•	-pi*x3**2* x4 - 4/3*pi*x3**3 <= -1296000  & x4 >=10 implique que x4 >= 1296000/pi*x3**2 - 4/3*x3 (x3!=0) & x4 >= 10

ce qui implique que : x4 >= max(1296000/pi*x3**2 - 4/3*x3, 10) et on a x3 <= 240


In [1047]:
def bornes(x):
    return np.asarray([[1,99],[1,100],[1,min(0.625*x[1]/0.00954, 0.625*x[0]/0.0193, 200)],[max(10,1296000/(np.pi*x[2]**2)-4/3*x[2]),240]])

La fonction objective :

In [1048]:
def cout_soudure(x):
            return 1.7781*0.625*x[1]*x[2]**2 + 0.6224*0.625*x[0]*x[2]*x[3] + 3.1661*(0.625*x[0])**2*x[3] + 19.84*(0.625*x[0])**2*x[2]


Cette fonction permet de vérifier si le nouveau point génère respecte les contraintes ou pas en se servant de la fonction 'bornes'. 

In [1049]:
def contraintes(x):
    _bornes = bornes(x)
    for i in range(len(x)):
        if x[i] < _bornes[i,0] or x[i] > _bornes[i,1]:
            return False
    return True


La fonction d'initialisation du premier point pour les deux algorithmes du hill climbing et du recuit simulé et qui le génère en respectant les contraintes imposées en se servant de la fonction 'bornes'. 

In [1050]:
def initialiser_points():
    x = np.zeros(4)
    _bornes = bornes(x)
    while not contraintes(x):
            x[0]= np.random.uniform(low =  _bornes[0,0],high = _bornes[0,1])
            x[1]= np.random.uniform(low =  _bornes[1,0], high =  _bornes[1,1])
            _bornes = bornes(x)
            x[2]= np.random.uniform(low =  _bornes[2,0],high =  _bornes[2,1])
            _bornes = bornes(x)
            x[3]= np.random.uniform(low =  _bornes[3,0], high =  _bornes[3,1])

    return x



Cette fonction est pour générer une liste de n points pour l'algorithme aléatoire.

In [1051]:
def generer_points(n):
    x = np.zeros([n,4])
    for i in range(n):
            _bornes = bornes(x[i])
            
            while not contraintes(x[i]):
                
                    x[i,0]= np.random.uniform(low =  _bornes[0,0],high = _bornes[0,1])
                    x[i,1]= np.random.uniform(low =  _bornes[1,0], high =  _bornes[1,1])
                    _bornes = bornes(x[i])
                    x[i,2]= np.random.uniform(low =  _bornes[2,0],high =  _bornes[2,1])
                    _bornes = bornes(x[i])
                    x[i,3]= np.random.uniform(low =  _bornes[3,0], high =  _bornes[3,1])

    return x
   


La fonction 'mutation' est utilisée aussi pour les deux algorithmes du hill climbing et du recuit simulé. Elle permet de modifier une des variables choisies aléatoirement du candidat actuel. Elle retourne un nouveau point en fonction de l'ancien à condition qu'il respecte les contraintes.
Pour ce, j'ai utilisé une mutation trouvée dans une référence du cours qui se sert de la distribution gaussienne. En plus, si la fonction ne trouve pas un nouveau candidat acceptable pendant dix itérations, elle change la dimension à modifier.


In [1052]:
def mutation(candidat):
    _bornes = bornes(candidat)

    
    temp_array = candidat

    cpt=0
    while True: #do-while
        
        if(cpt%10 == 0):
            i = np.random.randint(0,4)
            temp=candidat[i]
        
        temp = candidat[i] + np.random.normal(0,1) * 0.01 * (_bornes[i,1] - _bornes[i,0]) 
        temp_array[i] = temp
        
       

        if contraintes(temp_array): 
            candidat[i] = temp
            break
        
        cpt+=1
    return candidat


Les différentes heuristiques de refroidissement pour le recuit simulé:

In [1053]:
def calculer_temperature_expo(t, T_init,epsilon_t=0.025): #exponentiel
    return (1-epsilon_t)**t*T_init



In [1054]:
def calculer_temperature_log(t, T_init): # logarithmique
    if t<3:
        return T_init
    return T_init/np.log(t)


In [1055]:
def calculer_temperature_poly(t,T_init,t_max=200, alpha=2): # polynomial
    return (1-t/t_max)**alpha * T_init # t_max est l'iteration maximale pour laquelle la temperature doit etre nulle


Fonction de calcul de la probalbilite d'acceptation d'un nouveau point dont le score est plus mauvais que celui du point atuel.

In [1056]:
def proba_acceptance(T,deltaE):
    return np.exp(-deltaE/T)

'print_results' permet d'afficher les meilleurs résultats trouvés. 

In [1057]:
def print_results(hist):
    
    scores = hist[:,1]
    best_score = min(scores)
    index = np.where(scores == best_score)
    #print(f"index : {index}")
    print(f"meilleurs dimensions : \n z1 = {0.625*hist[index[0][0],3]} \n z2 = {0.625*hist[index[0][0],4]} \n x3 = {hist[index[0][0],5]} \n x4 = {hist[index[0][0],6]}\n")
    print(f"meilleur score : {best_score}\n")

   



Cette fonction prépare les données pour la présentation dans un tableau a l'aide de la bibliothèque 'pandas'. Elle permet de convertir une liste numpy en une DataFrame, avec le calcul de z1 et z2 à partir de x1 et x2.

In [1058]:
def preparation_table(hist):
    hist[:,3] = [0.625 * a for a in hist[:,3]]
    hist[:,4] = [0.625 * a for a in hist[:,4]]

    table = pd.DataFrame(data=hist[:,1:], columns=["Score","Nbr itrs", "z1","z2","x3","x4"], index=hist[:,0]) 
    
    return table




Fonction pour l'affichage de l'historique détaillé des points et scores.

In [1059]:
def print_table(hist):
      print("historique detaille :\n")
      return preparation_table(hist)

Fonction pour l'affichage d'une table contenant l'analyse de l'historique à l'aide de la méthode describe() de 'pandas'.

In [1060]:
def print_analyse(hist):
    print("Analyse des donnees generees :\n")
    return preparation_table(hist).describe()


L'algorithme de la recherche aléatoire : 

In [1061]:
def aleatoire(n = 50, itrs=100,epsilon = 0.001, samples_size = 10 ,verbose = False):
    
     
       #iterateur 
    i=0
    historique = np.empty([0,7])
    while(i<n):
        i+=1
        k=0
        l=0
        current_points = generer_points(samples_size)
        scores = np.apply_along_axis(cout_soudure, arr = current_points, axis = 1)
        score = np.min(scores)
        current_point = current_points[np.argmin(scores),]
        best_point = current_point
        best_score = score
        best_before_score = best_score
        
        while(k<itrs):
          k+=1   
          current_points = generer_points(samples_size)
          scores = np.apply_along_axis(cout_soudure, arr = current_points, axis = 1)

          score = np.min(scores)
          current_point = current_points[np.argmin(scores),]          
          
          best_before_score = best_score

          if(score < best_score):
                best_score = score
                best_point = current_point
          

          if best_before_score - best_score < epsilon :
                 l += 1 
          else :
                 l = 0

          if l == 50 : 
              break
          

          if verbose :
                print(f"itr : {k}, minimum : {current_point}, best score : {score}")
        
        historique = np.append(historique,[np.append([i,best_score,k],[best_point])],axis=0)
    return historique

In [1062]:
while True:
    iter = input("Entrer le nombre d'iterations : ")
    epsilon = input("Entrer epsilon ( progres minimal ) : ")

    if int(iter) > 0 and float(epsilon) > 0:
      break
    else:
      print("Veuillez reesayer!")
    
hist = aleatoire(itrs = int(iter), epsilon=float(epsilon))

<ipython-input-1047-6b33492640bc>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return np.asarray([[1,99],[1,100],[1,min(0.625*x[1]/0.00954, 0.625*x[0]/0.0193, 200)],[max(10,1296000/(np.pi*x[2]**2)-4/3*x[2]),240]])


In [1063]:
print_results(hist)

meilleurs dimensions : 
 z1 = 1.3767210474805958 
 z2 = 1.939598125313187 
 x3 = 60.0784197387637 
 x4 = 108.86495743316941

meilleur score : 20964.93791479527



In [1064]:
print_table(hist)

historique detaille :



,Score,Nbr itrs,z1,z2,x3,x4
1.0,26446.032531,10.0,1.963551,1.233420,56.208493,188.101222
2.0,82574.232496,10.0,5.043144,2.109175,89.348620,20.914695
3.0,27213.196844,10.0,2.311289,1.469481,74.252881,39.903838
4.0,71928.021815,10.0,3.010379,11.104022,49.110910,128.222285
5.0,44269.077448,10.0,3.282426,2.989003,42.422749,212.249387
6.0,264220.702670,10.0,3.957039,11.552569,92.223019,219.952712
7.0,82847.259180,10.0,2.059841,4.511283,92.772015,45.322374
8.0,27497.102975,10.0,2.009807,3.970352,38.092822,234.950957
9.0,103060.435245,10.0,1.246734,14.609055,60.445743,121.309502
10.0,24153.032737,10.0,0.964143,6.403787,38.980109,232.882345


In [1065]:
print_analyse(hist)

Analyse des donnees generees :



,Score,Nbr itrs,z1,z2,x3,x4
count,50.000000,50.0,50.000000,50.000000,50.000000,50.000000
mean,97004.653787,10.0,2.111111,6.325632,58.958800,156.018834
std,64174.407375,0.0,1.451910,6.441052,18.637848,67.048378
min,20964.937915,10.0,0.602589,0.419942,38.092822,20.914695
25%,44702.439804,10.0,1.206411,1.790768,43.766434,99.601652
50%,82710.745838,10.0,1.686510,3.749472,53.451675,170.389903
75%,133845.145194,10.0,2.479839,8.546846,69.966626,218.026881
max,271784.532353,10.0,7.684228,27.927596,118.083729,238.388301


L'algorithme du Hill Climbing : 

In [1066]:
def hill_climbing(itrs = 100,n = 50, epsilon = 0.001, verbose = False):
    i=0
    historique = np.empty([0,7])

    while(i<n):
      i+=1
      currentPoint = initialiser_points()
      bestScore = cout_soudure(currentPoint)
      bestPoint = currentPoint
      beforeBestScore = bestScore
      k=0 #iterateur 
      l=0

      while(k<itrs):

          k+=1   
          currentPoint = mutation(currentPoint) 
          score = cout_soudure(currentPoint)
          
          beforeBestScore = bestScore

          if(score < bestScore):
                bestScore = score
                bestPoint = currentPoint

          if beforeBestScore - bestScore < epsilon :
                 l += 1 
          else :
                 l = 0

          if l == 50 : 
              break
          

          if verbose :
                print(f"itr : {k}, minimum : {currentPoint},  score : {score}")
    
      historique = np.append(historique,[np.append([i,bestScore,k],[bestPoint])],axis=0)

    return historique

In [1067]:
while True:
    iter = input("Entrer le nombre d'iterations : ")
    epsilon = input("Entrer epsilon ( progres minimal ) : ")

    if int(iter) > 0 and float(epsilon) > 0:
      break
    else:
      print("Veuillez reesayer!")
hist = hill_climbing(itrs = int(iter), epsilon=float(epsilon))

<ipython-input-1047-6b33492640bc>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return np.asarray([[1,99],[1,100],[1,min(0.625*x[1]/0.00954, 0.625*x[0]/0.0193, 200)],[max(10,1296000/(np.pi*x[2]**2)-4/3*x[2]),240]])


In [1068]:
print_results(hist)

meilleurs dimensions : 
 z1 = 6.244385214150605 
 z2 = 22.640096624264018 
 x3 = 51.41133320591536 
 x4 = 237.40994015593253

meilleur score : 155288.89816842816



In [1069]:
print_table(hist)

historique detaille :



,Score,Nbr itrs,z1,z2,x3,x4
1.0,1.360817e+07,10.0,61.512783,2.757661,166.057511,73.344507
2.0,1.318264e+06,10.0,12.877405,50.325956,97.069701,127.850539
3.0,5.726143e+06,10.0,52.655890,46.837045,75.276884,102.174104
4.0,4.820925e+06,10.0,56.175399,32.031550,66.196113,42.010798
5.0,4.661274e+06,10.0,33.288750,47.794660,112.638772,229.987234
6.0,5.673072e+06,10.0,31.278239,12.739242,187.959317,188.643330
7.0,7.041448e+06,10.0,46.945765,21.496941,130.269742,64.602038
8.0,4.752305e+06,10.0,44.504272,7.636775,92.039076,137.686750
9.0,7.029334e+06,10.0,47.443259,19.827643,119.821431,109.922664
10.0,5.101117e+05,10.0,4.657935,43.617987,75.700142,114.813393


In [1070]:
print_analyse(hist)

Analyse des donnees generees :



,Score,Nbr itrs,z1,z2,x3,x4
count,5.000000e+01,50.0,50.000000,50.000000,50.000000,50.000000
mean,4.289216e+06,10.0,18.922706,19.863328,110.404203,131.834583
std,3.745407e+06,0.0,11.562295,10.328251,42.335135,63.999823
min,1.552889e+05,10.0,1.547273,1.723538,44.286324,15.740485
25%,1.250675e+06,10.0,8.416949,12.255103,82.250609,76.699855
50%,3.418576e+06,10.0,18.590529,20.100855,97.666761,127.291310
75%,6.322071e+06,10.0,29.576899,29.040608,135.406892,183.674629
max,1.482733e+07,10.0,38.445490,39.015523,199.770742,239.925620


L'algorithme du recuit simulé : 
Après chaque m itérations, on ajoute un nombre aléatoire par la loi uniforme entre 0 et 1. Par défaut, m est fixe à 10 itérations.


In [1071]:
def recuit_simule(T_init = 400,temperature= calculer_temperature_poly,itrs= 100,n = 50, m=10,epsilon = 0.001, verbose = False):
      # T_init ne doit pas etre posee comme ça
      
    
    historique = np.empty([0,7])
    i=0
    while(i<n):
        i+=1
        current_point = initialiser_points()
        best_point = current_point
        current_score = cout_soudure(current_point)
        best_score = current_score
        bestc_before_score = best_score
        k=0 # iterateur 
        l=0 # iterations pour suivre l'amelioration
        while(k<itrs):
          k+= 1
          
          new_point = mutation(current_point)
          new_score = cout_soudure(new_point)
          
          deltaE = new_score - current_score
          
          best_before_score = best_score

          if  deltaE <= 0 :   
              current_point = new_point
              current_score = new_score
              if current_score < best_score:
                  best_point = current_point
                  best_score = current_score
                
          else :
              T = temperature(k,T_init) + k//m * np.random.uniform(0,1) # Rechauffement apres chaque m iterations
              if np.random.uniform(0,1) < proba_acceptance(T,deltaE):
                  current_point = new_point
                  current_score = new_score
  

          if best_before_score - best_score < epsilon :  #peu ou pas d'amelioration 
                 l += 1 
          else :
                 l = 0

          if l == 50 : 
              current_point = best_point         #Retour au dernier meilleur point
              current_score = best_score
          

          if verbose :
                print(f"itr : {k}, point : {current_point}, score : {current_score}")
                
        historique = np.append(historique,[np.append([i,best_score,k],[best_point])],axis=0)

    return historique

Resultats pour le refroidissement polynomial : 

In [1072]:
while True:
    iter = input("Entrer le nombre d'iterations : ")
    epsilon = input("Entrer epsilon ( progres minimal ) : ")

    if int(iter) > 0 and float(epsilon) > 0:
      break
    else:
      print("Une des valeurs entrees n'est pas acceptee!")
    
hist = recuit_simule(itrs = int(iter), epsilon=float(epsilon))

<ipython-input-1047-6b33492640bc>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return np.asarray([[1,99],[1,100],[1,min(0.625*x[1]/0.00954, 0.625*x[0]/0.0193, 200)],[max(10,1296000/(np.pi*x[2]**2)-4/3*x[2]),240]])


In [1073]:
print_results(hist)

meilleurs dimensions : 
 z1 = 6.051209621058784 
 z2 = 10.803206806083788 
 x3 = 65.57144963126603 
 x4 = 218.92439493477983

meilleur score : 205109.66857755402



In [1074]:
print_table(hist)

historique detaille :



,Score,Nbr itrs,z1,z2,x3,x4
1.0,1.045029e+07,10.0,40.444984,40.659709,193.936377,145.707198
2.0,6.399583e+06,10.0,41.185731,50.763870,119.058380,141.068556
3.0,2.171174e+06,10.0,24.774792,42.178508,93.266460,119.811462
4.0,4.897634e+06,10.0,38.143778,27.107329,135.821421,25.499086
5.0,7.550020e+05,10.0,2.926296,30.044106,119.757988,50.793648
6.0,9.296921e+06,10.0,57.058660,33.944276,94.320470,195.317029
7.0,6.060461e+06,10.0,36.955314,15.989191,174.578577,57.313538
8.0,5.420722e+06,10.0,48.420753,50.394747,72.150098,177.309242
9.0,1.024110e+07,10.0,45.844159,48.677662,172.589338,82.013695
10.0,8.532889e+06,10.0,48.966956,22.821805,125.321643,175.606646


In [1075]:
print_analyse(hist)

Analyse des donnees generees :



,Score,Nbr itrs,z1,z2,x3,x4
count,5.000000e+01,50.0,50.000000,50.000000,50.000000,50.000000
mean,5.078177e+06,10.0,20.118630,23.733494,118.966916,134.433491
std,4.108354e+06,0.0,11.267012,10.057414,47.845463,70.251724
min,2.051097e+05,10.0,1.477506,0.707171,41.216824,13.218691
25%,1.560236e+06,10.0,11.814304,17.047354,73.482137,63.488577
50%,4.245577e+06,10.0,20.579866,24.601856,120.539368,151.481277
75%,8.257356e+06,10.0,30.116449,31.630544,154.641536,193.977373
max,1.635748e+07,10.0,38.315891,39.026779,199.947280,238.367261


Resultats pour le refroidissement exponentiel : 

In [1076]:
while True:
    iter = input("Entrer le nombre d'iterations : ")
    epsilon = input("Entrer epsilon ( progres minimal ) : ")

    if int(iter) > 0 and float(epsilon) > 0:
      break
    else:
      print("Une des valeurs entrees n'est pas acceptee!")
    
hist = recuit_simule(temperature = calculer_temperature_expo,itrs = int(iter), epsilon=float(epsilon))

<ipython-input-1047-6b33492640bc>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return np.asarray([[1,99],[1,100],[1,min(0.625*x[1]/0.00954, 0.625*x[0]/0.0193, 200)],[max(10,1296000/(np.pi*x[2]**2)-4/3*x[2]),240]])


In [1077]:
print_results(hist)

meilleurs dimensions : 
 z1 = 1.5666855005764102 
 z2 = 1.4619529692213185 
 x3 = 78.79538203279509 
 x4 = 236.98973914469838

meilleur score : 33569.129424975254



In [1078]:
print_table(hist)

historique detaille :



,Score,Nbr itrs,z1,z2,x3,x4
1.0,2.300645e+06,10.0,37.046451,21.966084,44.432158,196.210233
2.0,2.993442e+06,10.0,12.682936,45.753551,168.052232,95.786368
3.0,7.989012e+06,10.0,34.139303,53.201009,167.474103,208.525453
4.0,6.079104e+06,10.0,53.630260,22.151267,74.401049,190.381696
5.0,1.219749e+06,10.0,9.361609,17.664856,160.234735,119.381867
6.0,5.543292e+06,10.0,37.153816,17.955182,167.426908,54.603844
7.0,3.913124e+06,10.0,36.496789,61.397690,92.199682,98.748188
8.0,1.389267e+06,10.0,13.795938,16.573594,143.052134,191.028324
9.0,7.469613e+06,10.0,49.731573,12.635006,146.377878,47.950934
10.0,9.704645e+05,10.0,21.041561,18.716166,57.440169,202.403447


In [1079]:
print_analyse(hist)

Analyse des donnees generees :



,Score,Nbr itrs,z1,z2,x3,x4
count,5.000000e+01,50.0,50.000000,50.000000,50.000000,50.000000
mean,5.238342e+06,10.0,19.753670,18.438781,130.590396,125.160842
std,4.665248e+06,0.0,10.964710,10.431272,44.583899,68.900597
min,3.356913e+04,10.0,0.979178,0.913721,44.432158,19.389830
25%,1.547035e+06,10.0,10.115110,10.908103,84.861325,58.364879
50%,3.707208e+06,10.0,21.118031,16.822754,139.918591,124.878802
75%,7.383056e+06,10.0,28.317333,26.712883,167.332483,190.866667
max,1.689567e+07,10.0,38.445325,38.959405,197.135768,239.119552


Resultats pour le refroidissement logarithmique : 

In [1080]:
while True:
    iter = input("Entrer le nombre d'iterations : ")
    epsilon = input("Entrer epsilon ( progres minimal ) : ")

    if int(iter) > 0 and float(epsilon) > 0:
      break
    else:
      print("Une des valeurs entrees n'est pas acceptee!")
    
hist = recuit_simule(temperature = calculer_temperature_log,itrs = int(iter), epsilon=float(epsilon))

<ipython-input-1047-6b33492640bc>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return np.asarray([[1,99],[1,100],[1,min(0.625*x[1]/0.00954, 0.625*x[0]/0.0193, 200)],[max(10,1296000/(np.pi*x[2]**2)-4/3*x[2]),240]])


In [1081]:
print_results(hist)

meilleurs dimensions : 
 z1 = 8.23389282173043 
 z2 = 4.6533976492847575 
 x3 = 56.81696257998691 
 x4 = 216.53680614608822

meilleur score : 212236.5168149154



In [1082]:
print_table(hist)

historique detaille :



,Score,Nbr itrs,z1,z2,x3,x4
1.0,3.044398e+06,10.0,21.733553,42.311529,128.845632,181.559850
2.0,5.819002e+06,10.0,27.204209,46.184176,164.101668,236.167178
3.0,3.593838e+06,10.0,24.968947,41.217912,145.136135,97.865534
4.0,8.052063e+06,10.0,55.454446,9.171618,101.326605,129.357000
5.0,6.519317e+05,10.0,17.299597,29.760628,51.161476,173.575762
6.0,1.301384e+07,10.0,58.315571,40.512082,148.831318,85.137100
7.0,3.537250e+06,10.0,36.327623,56.966349,97.793076,33.474073
8.0,6.453829e+05,10.0,14.104424,28.556903,72.736654,178.936080
9.0,2.122365e+05,10.0,8.233893,4.653398,56.816963,216.536806
10.0,7.924082e+05,10.0,6.946065,44.805888,91.648194,141.188756


In [1083]:
print_analyse(hist)

Analyse des donnees generees :



,Score,Nbr itrs,z1,z2,x3,x4
count,5.000000e+01,50.0,50.000000,50.000000,50.000000,50.000000
mean,4.687840e+06,10.0,19.680691,21.949935,111.201721,138.480597
std,4.149847e+06,0.0,11.240813,10.897171,44.876050,73.975693
min,2.122365e+05,10.0,1.927023,0.806093,40.040752,13.626230
25%,1.160040e+06,10.0,9.768869,13.227096,73.947475,82.196546
50%,3.565544e+06,10.0,18.333449,25.308615,106.736473,140.569897
75%,7.151424e+06,10.0,28.509929,30.259304,145.178556,215.112288
max,1.833070e+07,10.0,38.116972,38.017046,195.054868,236.167178


### Conclusion

En moyenne sur 50 exécutions des 3 algorithmes, l'algorithme aléatoire est celui qui a donné les meilleurs résultats ce qui est assez surprenant, en choisissant pour toutes les exécutions effectuées 10 itérations par cycle (50 cycles) et 0.01 pour epsilon. 

En conclusion, le meilleur coût trouvé est de 20964.93791479527.
